# 분류기 만들기

## 타이타닉 데이터의 생존여부 분류
- 규칙 : 성별(sex) = 1 생존하지 않은 것으로 분류

In [20]:
from sklearn.base import BaseEstimator
import numpy as np
class MyDummyClassifier(BaseEstimator):
    def fit(self, x, y):
        pass
    
    def predict(self, x):
        pred = np.zeros((x.shape[0],1))
        for i in range(x.shape[0]):
            if x['Sex'].iloc[i] == 1:
                pred[i]=0
            else :
                pred[i]=1
        return pred
    

## 타이타닉 데이터 가져오기

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
titanic_df = pd.read_csv('./data/titanic.csv')
y_titanic_df = titanic_df['Survived']
x_titanic_df = titanic_df.drop('Survived', axis=1)

In [22]:
from sklearn.preprocessing import LabelEncoder

#Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

#머신러닝 알고리즘에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

#레이블 인코딩 수행 함수
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

#앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df) 
    df = drop_features(df)
    df = format_features(df)
    return df

In [23]:
x_titanic_df = transform_features(x_titanic_df)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11524\3921744607.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(), inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11524\3921744607.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [24]:
# 데이터셋 분할
x_train, x_test, y_train, y_test = train_test_split(x_titanic_df,
                                                    y_titanic_df,
                                                    test_size=0.2,
                                                    random_state=0)

In [25]:
myclf = MyDummyClassifier()
myclf.fit(x_train,y_train)

In [32]:
my_pred = myclf.predict(x_test)
accuracy_score(y_test, my_pred)

0.7877094972067039

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, my_pred)

array([[92, 18],
       [20, 49]])

In [34]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_test, my_pred), recall_score(y_test,my_pred)

(np.float64(0.7313432835820896), np.float64(0.7101449275362319))

# 로지스틱회귀, 랜덤포레스트, KNN의 정밀도, 재현율 비교하기

In [ ]:
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)

    print(confusion)
    print('*'*20)
    print(accuracy, precision, recall)

In [38]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(max_iter=2000)
lr_clf.fit(x_train,y_train)
pred = lr_clf.predict(x_test)

#정확도, 정밀도, 재현율
get_clf_eval(y_test, pred)

[[92 18]
 [16 53]]
--------------------
0.8100558659217877 0.7464788732394366 0.7681159420289855


In [43]:
pred_proba = lr_clf.predict_proba(x_test)
pos_proba = pred_proba[:,1] # 양성클래스일 확률

threshold = 0.4 #임계치
custom_proba = (pos_proba>=threshold).astype(int) # 임계치보다 크면 1
confusion_matrix(y_test, custom_proba)
get_clf_eval(y_test, custom_proba)

[[86 24]
 [13 56]]
--------------------
0.7932960893854749 0.7 0.8115942028985508


# 정밀도와 재현율의 변화
정밀도와 재현율의 불균형이 심할 때,
혹은 비즈니스의 요구사항이 있을 때
임계치를 조정해야한다.

임계치를 낮추면, 정밀도는 낮아지고, 재현율은 올라간다.

In [45]:
from sklearn.metrics import f1_score
f1_score(y_test, pred)

np.float64(0.7571428571428571)